In [ ]:
!pip install langchain openai chromadb gradio pypdf


In [ ]:
import os
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


In [ ]:
# 🔐 Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your actual key


In [ ]:
def process_pdf(file_path, persist_directory="chroma_db"):
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)

    embedding = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(chunks, embedding=embedding, persist_directory=persist_directory)
    return vectordb


In [ ]:
qa_chain = None


In [ ]:
def upload_pdf(file):
    global qa_chain
    vectordb = process_pdf(file.name)
    llm = ChatOpenAI(model_name="gpt-3.5-turbo")
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectordb.as_retriever())
    return "✅ PDF processed. You can now ask questions!"

def ask_question(question):
    if qa_chain is None:
        return "❌ Please upload and process a PDF first."
    return qa_chain.run(question)


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📄 Chat with your PDF (OpenAI + LangChain)")

    with gr.Row():
        file_input = gr.File(label="Upload PDF")
        upload_btn = gr.Button("Process PDF")
        status = gr.Textbox(label="Status")

    upload_btn.click(fn=upload_pdf, inputs=[file_input], outputs=[status])

    with gr.Row():
        query = gr.Textbox(label="Your Question")
        ask_btn = gr.Button("Ask")
        answer = gr.Textbox(label="Answer")

    ask_btn.click(fn=ask_question, inputs=[query], outputs=[answer])

demo.launch()
